In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://github.com/achireistefan/Pothole-Detection


Cloning into 'Pothole-Detection'...
remote: Enumerating objects: 27205, done.
remote: Total 27205 (delta 0), reused 0 (delta 0), pack-reused 27205
Receiving objects: 100% (27205/27205), 1.61 GiB | 14.02 MiB/s, done.
Resolving deltas: 100% (11251/11251), done.
Updating files: 100% (36187/36187), done.


In [ ]:
!ls Pothole-Detection


Pothole_detection_voc  Pothole_detection_yolo  README.md  sov_dataset_comparison


In [ ]:
!ls Pothole-Detection/Pothole_detection_voc
!ls Pothole-Detection/Pothole_detection_yolo
!ls Pothole-Detection/sov_dataset_comparison


Annotations  ImageSets	JPEGImages  labels.txt
ReadMe.txt  test  train_augmented  train_original
sov_dataset  sov_dataset_output_new  sov_dataset_output_old


In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

# Example to load trainval.txt
trainval_path = '/content/Pothole-Detection/Pothole_detection_voc/ImageSets/Main/trainval.txt'
trainval_images = pd.read_csv(trainval_path, header=None, names=['filename'])

# Example to parse an XML annotation
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for obj in root.iter('object'):
        print(obj.find('name').text)

# Test parse_annotation with an example XML file
example_xml = '/content/Pothole-Detection/Pothole_detection_voc/Annotations/example.xml'  # Adjust this path
parse_annotation(example_xml)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Pothole-Detection/Pothole_detection_voc/Annotations/example.xml'

In [ ]:
!ls /content/Pothole-Detection/Pothole_detection_voc/Annotations


 0_0_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_1_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_2_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_3_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_4_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_5_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_6_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
 0_7_img-47_jpg.rf.53aba5861cc3e2c2f7783276288b2b97.xml
'1000_0_left000019 - Copy.xml'
'1000_1_left000019 - Copy.xml'
'1000_2_left000019 - Copy.xml'
'1000_3_left000019 - Copy.xml'
'1000_4_left000019 - Copy.xml'
'1000_5_left000019 - Copy.xml'
'1000_6_left000019 - Copy.xml'
'1000_7_left000019 - Copy.xml'
 100_0_left001085.xml
 1001_0_p07.xml
 1001_1_p07.xml
 1001_2_p07.xml
 1001_3_p07.xml
 1001_4_p07.xml
 1001_5_p07.xml
 1001_6_p07.xml
 1001_7_p07.xml
 100_1_left001085.xml
 1002_0_img-95_jpg.rf.88e456640460ae2424d7881caa5d1f86.xml
 1002_1_img-95_jpg.rf.88e456640460ae2424d7881caa5d1f86.xml
 1002_2_img-95

In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        # Initialize the layers of the Generator
        self.init_layers = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 1024, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True)
        )

        self.middle_layers = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )

        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(256, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        # Assuming z is reshaped properly before being fed into the network
        x = z.view(-1, self.latent_dim, 1, 1)  # Reshape z to (batch_size, latent_dim, 1, 1)
        x = self.init_layers(x)
        x = self.middle_layers(x)
        x = self.final_layer(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.final_layer = None
        self.num_features = None  # Store the number of features to detect changes

    def forward(self, img):
        features = self.conv_layers(img)
        current_num_features = features.shape[1] * features.shape[2] * features.shape[3]

        if self.final_layer is None or self.num_features != current_num_features:
            self.num_features = current_num_features
            self.final_layer = nn.Sequential(
                nn.Flatten(),
                nn.Linear(self.num_features, 1),
                nn.Sigmoid()
            ).to(features.device)
            print(f"Reinitialized final_layer to accommodate feature size: {self.num_features}")

        validity = self.final_layer(features)
        return validity





In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, io
from torchvision.utils import save_image
import numpy as np
import torch.nn as nn
import xml.etree.ElementTree as ET

class VOCDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images_dir = os.path.join(root_dir, 'JPEGImages')
        self.annotations_dir = os.path.join(root_dir, 'Annotations')
        self.filenames = [os.path.splitext(file)[0] for file in os.listdir(self.images_dir) if file.endswith('.jpg')]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.filenames[idx] + '.jpg')
        image = io.read_image(img_name)
        if self.transform:
            image = self.transform(image)

        annotation_name = os.path.join(self.annotations_dir, self.filenames[idx] + '.xml')
        boxes = self.parse_annotations(annotation_name)
        sample = {'image': image, 'boxes': boxes}
        return sample

    def parse_annotations(self, file_path):
        tree = ET.parse(file_path)
        root = tree.getroot()
        boxes = []
        for member in root.findall('object'):
            xmin = int(member.find('bndbox/xmin').text)
            ymin = int(member.find('bndbox/ymin').text)
            xmax = int(member.find('bndbox/xmax').text)
            ymax = int(member.find('bndbox/ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
        return boxes

def my_collate_fn(batch):
    images = torch.stack([item['image'] for item in batch])
    max_boxes = max(len(item['boxes']) for item in batch)
    padded_boxes = [torch.tensor(item['boxes'] + [[0, 0, 0, 0]] * (max_boxes - len(item['boxes'])))
                    for item in batch]
    boxes = torch.stack(padded_boxes)
    return {'image': images, 'boxes': boxes}

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ensures all images are resized to the same dimensions
    transforms.ConvertImageDtype(torch.float),  # Convert image to float
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalizes the image
])

def get_data_loader(image_dir, batch_size):
    dataset = VOCDataset(root_dir=image_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)
    return dataloader

def train(discriminator, generator, criterion, optimizer_g, optimizer_d, device, epochs, dataloader):
    for epoch in range(epochs):
        for i, data in enumerate(dataloader):
            real_imgs = data['image'].to(device)
            batch_size = real_imgs.size(0)
            valid = torch.ones(batch_size, 1, device=device)
            fake = torch.zeros(batch_size, 1, device=device)

            # Generate a batch of images
            z = torch.randn(batch_size, generator.latent_dim, 1, 1, device=device)
            gen_imgs = generator(z)

            # Train Discriminator
            optimizer_d.zero_grad()
            real_loss = criterion(discriminator(real_imgs), valid)
            fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_g.step()

            if i % 100 == 0:
                print(f"[Epoch {epoch}] [Batch {i}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
                save_image(gen_imgs.data[:25], f'images/{epoch}_{i}.png', nrow=5, normalize=True)

if __name__ == '__main__':
    image_dir = '/content/Pothole-Detection/Pothole_detection_voc'
    batch_size = 10
    dataloader = get_data_loader(image_dir, batch_size)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator = Generator(100).to(device)
    discriminator = Discriminator().to(device)
    criterion = nn.BCELoss()
    optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    train(discriminator, generator, criterion, optimizer_g, optimizer_d, device, 25, dataloader)


Streaming output truncated to the last 5000 lines.
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accommodate feature size: 131072
Reinitialized final_layer to accommodate feature size: 2048
Reinitialized final_layer to accomm

In [ ]:
import os

# Example to list contents of the directory
image_dir = '/content/Pothole-Detection'
print(os.listdir(image_dir))  # Check what directories/files are in your specified path


['README.md', 'Pothole_detection_voc', '.git', 'sov_dataset_comparison', 'Pothole_detection_yolo', '.ipynb_checkpoints']


In [ ]:
sub_dir = os.path.join(image_dir, 'Pothole_detection_voc')
print(os.listdir(sub_dir))  # This will show the contents of 'Pothole_detection_voc'


['JPEGImages', 'labels.txt', 'ImageSets', 'Annotations']


In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, io
from torchvision.utils import save_image
import numpy as np
import xml.etree.ElementTree as ET

class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.init_layers = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 1024, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True)
        )
        self.middle_layers = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)
        )
        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(256, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(-1, self.latent_dim, 1, 1)
        x = self.init_layers(z)
        x = self.middle_layers(x)
        x = self.final_layer(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.final_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, 1),  # Adjust based on the output size from conv_layers
            nn.Sigmoid()
        )

    def forward(self, img):
        features = self.conv_layers(img)
        validity = self.final_layer(features)
        return validity

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resizes the image
    transforms.ConvertImageDtype(torch.float),  # Converts image to float type
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalizes the tensor image
])

class VOCDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images_dir = os.path.join(root_dir, 'JPEGImages')
        self.annotations_dir = os.path.join(root_dir, 'Annotations')
        self.filenames = [os.path.splitext(file)[0] for file in os.listdir(self.images_dir) if file.endswith('.jpg')]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.filenames[idx] + '.jpg')
        image = io.read_image(img_name)  # Returns a tensor
        if self.transform:
            image = self.transform(image)

        annotation_name = os.path.join(self.annotations_dir, self.filenames[idx] + '.xml')
        boxes = self.parse_annotations(annotation_name)
        sample = {'image': image, 'boxes': boxes}
        return sample

    def parse_annotations(self, file_path):
        tree = ET.parse(file_path)
        root = tree.getroot()
        boxes = []
        for member in root.findall('object'):
            xmin = int(member.find('bndbox/xmin').text)
            ymin = int(member.find('bndbox/ymin').text)
            xmax = int(member.find('bndbox/xmax').text)
            ymax = int(member.find('bndbox/ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
        return boxes

# Ensure transformations are compatible with tensor inputs
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resizes the image
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalizes the tensor image
])

def get_data_loader(image_dir, batch_size):
    dataset = VOCDataset(root_dir=image_dir, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

def train(discriminator, generator, criterion, optimizer_g, optimizer_d, device, epochs, dataloader):
    for epoch in range(epochs):
        for i, data in enumerate(dataloader):
            real_imgs = data['image'].to(device)
            batch_size = real_imgs.size(0)
            valid = torch.ones(batch_size, 1, device=device)
            fake = torch.zeros(batch_size, 1, device=device)

            # Generate a batch of images
            z = torch.randn(batch_size, generator.latent_dim, 1, 1, device=device)
            gen_imgs = generator(z)

            # Train Discriminator
            optimizer_d.zero_grad()
            real_loss = criterion(discriminator(real_imgs), valid)
            fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            g_loss = criterion(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_g.step()

            if i % 100 == 0:
                print(f"[Epoch {epoch}] [Batch {i}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
                save_image(gen_imgs.data[:25], f'images/{epoch}_{i}.png', nrow=5, normalize=True)

if __name__ == '__main__':
    image_dir = '/content/Pothole-Detection/Pothole_detection_voc'
    batch_size = 10
    dataloader = get_data_loader(image_dir, batch_size)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator = Generator(100).to(device)
    discriminator = Discriminator().to(device)
    criterion = nn.BCELoss()
    optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    train(discriminator, generator, criterion, optimizer_g, optimizer_d, device, 25, dataloader)


TypeError: Input tensor should be a float tensor. Got torch.uint8.

In [ ]:
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1)  # No sigmoid activation
        )

    def forward(self, img):
        return self.model(img)


In [ ]:
def compute_gradient_penalty(critic, real_samples, fake_samples, device):
    """Calculates the gradient penalty loss for WGAN GP"""
    alpha = torch.rand((real_samples.size(0), 1, 1, 1), device=device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = critic(interpolates)
    fake = torch.autograd.Variable(torch.FloatTensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False, device=device)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty


In [ ]:
from torchvision.utils import save_image
import os
images_dir = 'generated_images'
os.makedirs(images_dir, exist_ok=True)


In [ ]:
def train(critic, generator, criterion, optimizer_g, optimizer_d, device, epochs, dataloader):
    lambda_gp = 10  # Gradient penalty lambda hyperparameter
    for epoch in range(epochs):
        for i, data in enumerate(dataloader):
            real_imgs = data['image'].to(device)
            batch_size = real_imgs.size(0)

            # Training the Critic
            optimizer_d.zero_grad()
            z = torch.randn(batch_size, generator.latent_dim, 1, 1, device=device)
            fake_imgs = generator(z).detach()
            real_validity = critic(real_imgs)
            fake_validity = critic(fake_imgs)
            gradient_penalty = compute_gradient_penalty(critic, real_imgs.data, fake_imgs.data, device)
            d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
            d_loss.backward()
            optimizer_d.step()

            # Train the Generator every n_critic iterations
            if i % 5 == 0:
                optimizer_g.zero_grad()
                gen_imgs = generator(z)
                g_loss = -torch.mean(critic(gen_imgs))
                g_loss.backward()
                optimizer_g.step()

            if i % 100 == 0:
                print(f"[Epoch {epoch}] [Batch {i}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
                save_image(gen_imgs.data[:25], f'{images_dir}/{epoch}_{i}.png', nrow=5, normalize=True)

        # Optionally save images at the end of each epoch
        save_image(gen_imgs.data[:25], f'{images_dir}/epoch_{epoch}_final.png', nrow=5, normalize=True)
